In [202]:
import praw
import os
import configparser
import webbrowser
import random
import json
import pandas as pd
import numpy as np
import collections
import string
from nltk.corpus import stopwords
from nltk.stem import *
os.chdir("C:/Users/macle/Desktop/UPC Masters/Semester 2/CI/Final Project")

In [128]:
config = configparser.ConfigParser()
config.read('secrets.ini')

reddit_client_id = config.get('reddit', 'client_id')
reddit_api_key = config.get('reddit', 'api_key')
r.set_oauth_app_info(client_id=reddit_client_id,
                      client_secret=reddit_api_key,
                      redirect_uri='http://cole-maclean.github.io/MAI-CI/')

In [129]:
url = r.get_authorize_url('uniqueKey', 'identity', True)
webbrowser.open(url)

True

In [130]:
access_information = r.get_access_information('B3w4W9k5xFbJpaLv8Xpuf-iQj34')
r.set_access_credentials(**access_information)

In [131]:
authenticated_user = r.get_me()
print(authenticated_user.name, authenticated_user.link_karma)

upcmaici 1


In [235]:
rand_submission = r.get_random_submission()
rand_submission.title

'[Battlefield 1] Surprise attack!!'

In [ ]:
with open('reddit_data.json','r') as data_file:    
    reddit_data = json.load(data_file)
with open('scrapped_users.json','r') as data_file:    
    scrapped_users = json.load(data_file)
for i in range(10):
    print ("scrapping " + str(i) + "th subreddit")
    rand_submission = r.get_random_submission()
    print ("scrapping users from subreddit r/" + rand_submission.subreddit.display_name)
    if len(rand_submission.comments) >=3:
        rnd_comments = random.sample(rand_submission.comments,min(10,len(rand_submission.comments)))
        for comment in rnd_comments:
            if isinstance(comment, praw.objects.Comment):
                user = comment.author
                if user:
                    if user.name in scrapped_users:
                        print ('user ' + user.name + 'already scraped')
                    else:
                        scrapped_users.append(user.name)
                        for user_comment in user.get_comments(limit=None,_use_oauth=False):
                            body = user_comment.body.split()
                            if len(body) >= 10:
                                rand_words = random.sample(body,10)
                            else:
                                rand_words = body
                            reddit_data.append([user.name,user_comment.subreddit.display_name,rand_submission.title,
                                                                              user_comment.created_utc,rand_words])
                    
with open('reddit_data.json','w') as data_file:
    json.dump(reddit_data, data_file)
with open('scrapped_users.json','w') as data_file:
    json.dump(scrapped_users, data_file)

scrapping 0th subreddit
scrapping users from subreddit r/learnpython
scrapping 1th subreddit
scrapping users from subreddit r/titanfall
scrapping 2th subreddit
scrapping users from subreddit r/DotA2
scrapping 3th subreddit
scrapping users from subreddit r/BigBoobsGW

In [212]:
list_of_words = []
translate_table = dict((ord(char), None) for char in string.punctuation)
stop = set(stopwords.words('english'))
stemmer = PorterStemmer()
for user_comment in reddit_data:
    for word in user_comment[4]:
        if len(word) < 45 and word not in stop:
            clean_word = word.translate(translate_table)
            wrd_stm = stemmer.stem(clean_word)
            list_of_words.append(wrd_stm)
word_counts = collections.Counter(list_of_words)

In [207]:
df = pd.DataFrame(reddit_data,columns=['user','subreddit','submission','utc_stamp','rnd_words'])
df.head()

,user,subreddit,utc_stamp,rnd_words
0,CrossBreedP,gaming,1479542844,"[DILDO!, mummy, KITTY'S, a, kitty, with, A, to..."
1,CrossBreedP,tifu,1479493301,"[1994, was, technically, 22, years, ago]"
2,CrossBreedP,AskReddit,1479489063,"[as, I, concert, my, day, Best, tour, life., t..."
3,CrossBreedP,AskReddit,1479436213,"[messages"", thinking, of, the, customer.), amm..."
4,CrossBreedP,AskReddit,1479405262,"[of, who, she, bitch, always, went, She, her, ..."


In [208]:
f = {'subreddit':['count'], 'utc_stamp':[np.min,np.max],'rnd_words': 'sum'}
grouped = df.groupby(['user','subreddit']).agg(f)

In [226]:
grouped[['rnd_words','utc_stamp']]

rnd_words  \
                                                                                  sum   
user          subreddit                                                                 
2honourable4u AskReddit             [year)., grade, late, in, girl, I, shit, in, a...   
              Gunners               [Na, na, naaaaaaaaa..., and, forth, and, so, n...   
              KissAnime             [few, instead, fake, fake, a, a, It's, one, th...   
              NoMansSkyTheGame      [Yeah,, man,, I, know..., Holy, shit..., I'M, ...   
              Showerthoughts        [true, of, this, is, angry, how, I'm, at, myse...   
              WTF                        [He, might, want, to, go, see, a, doctor...]   
              anime                 [stop, knew, just, a, name, had, I, there, be,...   
              aww                                                 [FUCK,, my, heart!]   
              dataisbeautiful                        [Female, entrepreneurs., Sweet!]   
              gaming                       [Cowabunga,, dude!, Look, at, him, squirm]   
              mildlyinteresting     [This, hurts, my, delicate, architectural, min...   
              nottheonion                                        [By, Fall, Out, Boy]   
              shittyaskscience      [normal., has, no,, as, not, actually, someone...   
              soccer                [To, me,, it, looked, like, Trippier.....tripp...   
              sports                [a, There's, name, heard, same, player, the, a...   
BodetGrrla    AUTOMOBILISTA         [One, Turn, /, Multi, Class, @, Kensai, (Suzuk...   
              Bitcoin               [I, who, so, play, boot, with, up, promising, ...   
              Cyberpunk             [animation, :-D, her, paste,, light, are, emit...   
              Documentaries         [or, the, next, Nokia, range)., with, 1, kW, t...   
              Drifting              [Assetto, Corsa, http://imgur.com/gallery/cGAh...   
              Futurology                                                     [Berlin]   
              InternetIsBeautiful   [with, different, sound, https://www.youtube.c...   
              OldSchoolCool                                          [>, ͡°, ͜ʖ, ͡°<]   
              assettocorsa          [of, my, lightweight, 113GB, 1.2GB, AC, is, ri...   
              atheism               ["I, love, criminals"???, WTF, a, not, put, gr...   
              books                 [abstract, for, of, mind?, cut, in, it, to, do...   
              exmuslim              [the, all, HL3, confirmed, be, 1, 3., prayers,...   
              gaming                [VR, physical, physical, needed,, --, --, leve...   
              janusVR               [thx, a, lot,, works, like, a, charm, anybody,...   
              oculus                [Mazda, just, for, are, around., Roadster, fee...   
...                                                                               ...   
xabulba       no_sob_story                                     [pears, or, avocados?]   
              nottheonion           [That's, racist,, white, chicks, like, menthol...   
              obama                 [annual, Dog, left, barren,, factory, there, P...   
              oddlysatisfying       [to, Why, would, they, what, aren't, trying, t...   
              pics                  [http://wallpaper4k.info/wp-content/uploads/20...   
              politics              [a, their, naming, to, tried, administration.,...   
              popping                                           [I, read, pooping...]   
              rage                  [for, acting, was, love, I, of, is, their, inj...   
              raisedbynarcissists                              [No,, no, you, don't.]   
              rpg                   [Shouldn't, that, last, one, be, Project, Shad...   
              science               [TIL, Great, White, sharks, poop, green., Only...   
              scifi                 [Dark, parallels, it?, Makes, wonde